In [1]:
# Import required libraries for scrapping
import urllib.request
from bs4 import BeautifulSoup
import smtplib
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Regular Expressions to collect the addresses
import re


#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values



# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Class object has been created to store the credentials for safekeeping
#import credentials_oa_outlk

print('Libraries imported.')

Libraries imported.


In [2]:
# Set/access url to be scrapped
url = 'https://www.immobilienscout24.de/Suche/radius/wohnung-mieten?centerofsearchaddress=D%C3%BCsseldorf;;;1276010012;Nordrhein-Westfalen;&numberofrooms=2.0-&price=-2000.0&geocoordinates=51.23824;6.81513;3.0&pagenumber=3'
headers = {"User-agent": 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

page = requests.get(url, headers=headers)

# Parsing through the URL
soup = BeautifulSoup(page.content, 'html.parser')

print("Data parsed.")

Data parsed.


## ~~Getting the address~~

In [3]:
# Getting the address of each house
address  = soup.find_all('button')

address1 = []
for a in address:
    a = a.decode()
    stuff = re.findall(("[^<>]+\w+[A-Za-z].\\s+Düsseldorf"), a)
    if len(stuff) == 0:
        continue
    else:        
        address1.append(stuff)
   
#address1

## ~~Getting the price~~

In [4]:
# Getting the price for each house
price  = soup.find_all('dd')

price1 = []
for p in price:
    p = p.decode()
    stuff = re.findall(("[^<>]+\\s+€"),p)
    if len(stuff) == 0:
        continue
    else:        
        price1.append(stuff)

#price1

## ~~Getting the number of rooms~~

In [5]:
# Getting the number of rooms
rooms = soup.findAll(class_=['onlySmall'])

rooms1 = []
for r in rooms:
    r = r.decode()
    stuff = re.findall(("[>]+[0-9]+[<]"), r)
    if len(stuff) == 0:
        continue
    else:        
        rooms1.append(stuff)

# rooms1

In [6]:
"""
https://towardsdatascience.com/top-5-beautiful-soup-functions-7bfe5a693482
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

https://mkyong.com/computer-tips/how-to-view-http-headers-in-google-chrome/

"""

'\nhttps://towardsdatascience.com/top-5-beautiful-soup-functions-7bfe5a693482\nhttps://www.crummy.com/software/BeautifulSoup/bs4/doc/\n\nhttps://mkyong.com/computer-tips/how-to-view-http-headers-in-google-chrome/\n\n'

## Getting the number of habitants per borough

In [7]:
url_hab = 'http://www.citypopulation.de/en/germany/dusseldorf/admin/'


In [8]:
# Getting the Coordinates for Düsseldorf

address = 'Düsseldorf, NRW'

geolocator = Nominatim(user_agent="duesseldorf_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Düsseldorf are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Düsseldorf are 51.2254018, 6.7763137.


## Getting the Boroughs

In [9]:
url_wiki = 'https://de.wikipedia.org/wiki/Liste_der_Stadtbezirke_von_D%C3%BCsseldorf'

page = requests.get(url_wiki)
# Parsing through the URL
soup = BeautifulSoup(page.content, 'html.parser')


print("Data parsed.")

Data parsed.


In [10]:
all_tables = soup.find_all("table")
right_table = soup.find('table', class_ = 'wikitable sortable')

# class="admin2" | class="rpop prio1"
# bors = soup.findAll(class_=['admin2'])

# 
### Assigning Districts and Boroughs to DataFrame

In [110]:
# SET THE DATAFRAME COLUMNS || CREATE EMPTY DATAFRAME WITH THE SAME STRUCTURE OF THE ORIGINAL
column_names = ['District','Borough','Fläche','Einwohner','Bewölkerungsdichte','Bezirkvorsteher','K','Karte']
df = pd.DataFrame(columns = column_names)

#####################################################
# LOAD OF THE DATAFRAME || CLEAR TO THE 2 COLUMNS NEEDEED
for tr_cell in right_table.find_all('tr'):
    row_data = []
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.rstrip())
        
    if len(row_data)>0:
        df.loc[len(df)] = row_data
        
df = df[['District', 'Borough']]
print(df.head())

#####################################################
# REMOVE NUMERIC DIGITS FROM THE BOROUGHS
df.set_index(['District', 'Borough'])  
df = df.replace(regex=r'[0-9]+', value=',')
district_numb = ['1','2','3','4','5','6','7','8','9','10']
df['district_numb'] = district_numb

print(df.head())

           District                                            Borough
0  Stadtbezirk 1[1]  011 Altstadt012 Carlstadt013 Stadtmitte014 Pem...
1  Stadtbezirk 2[2]     021 Flingern Süd022 Flingern Nord023 Düsseltal
2  Stadtbezirk 3[3]  031 Friedrichstadt032 Unterbilk033 Hafen034 Ha...
3  Stadtbezirk 4[4]  041 Oberkassel042 Heerdt043 Lörick044 Niederka...
4  Stadtbezirk 5[5]  051 Stockum052 Lohausen053 Kaiserswerth054 Wit...
           District                                            Borough  \
0  Stadtbezirk ,[,]  , Altstadt, Carlstadt, Stadtmitte, Pempelfort,...   
1  Stadtbezirk ,[,]           , Flingern Süd, Flingern Nord, Düsseltal   
2  Stadtbezirk ,[,]  , Friedrichstadt, Unterbilk, Hafen, Hamm, Volm...   
3  Stadtbezirk ,[,]         , Oberkassel, Heerdt, Lörick, Niederkassel   
4  Stadtbezirk ,[,]  , Stockum, Lohausen, Kaiserswerth, Wittlaer, A...   

  district_numb  
0             1  
1             2  
2             3  
3             4  
4             5  


### Check the Import part

In [111]:
import numpy as np
from itertools import chain

# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lens = df['Borough'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate
res = pd.DataFrame({'District': np.repeat(df['District'], lens),
                    'Borough': chainer(df['Borough']),
                    'district_numb': np.repeat(df['district_numb'], lens)})
# Strip empty cells ""
res = res[res['Borough'].str.strip().astype(bool)]

In [113]:
res.head()

,District,Borough,district_numb
0,"Stadtbezirk ,[,]",Altstadt,1
0,"Stadtbezirk ,[,]",Carlstadt,1
0,"Stadtbezirk ,[,]",Stadtmitte,1
0,"Stadtbezirk ,[,]",Pempelfort,1
0,"Stadtbezirk ,[,]",Derendorf,1


In [114]:
res

,District,Borough,district_numb
0,"Stadtbezirk ,[,]",Altstadt,1
0,"Stadtbezirk ,[,]",Carlstadt,1
0,"Stadtbezirk ,[,]",Stadtmitte,1
0,"Stadtbezirk ,[,]",Pempelfort,1
0,"Stadtbezirk ,[,]",Derendorf,1
0,"Stadtbezirk ,[,]",Golzheim,1
1,"Stadtbezirk ,[,]",Flingern Süd,2
1,"Stadtbezirk ,[,]",Flingern Nord,2
1,"Stadtbezirk ,[,]",Düsseltal,2
2,"Stadtbezirk ,[,]",Friedrichstadt,3
